In [2]:
# USD vs. GBP

### Part I : Peeling out the monetary shock

Docker = "To peel out a monetary shocks TS, the data needed are:\n \
          * The federal funds rate series \n \
          * The inflation sequence \n \
          * The output gap sequence."
"""To get the output gap sequence, we use a quadratic function to fit the trend and determine the gap."""
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
from matplotlib import pyplot as plt
%matplotlib inline
import statsmodels.api as sm

def flashBack(old,n,mode="month"):
    #Get the back the argued period of time
    if mode == "month":
        #Flash back by month
        year = old.year - int((n - n%12)/12)
        res = n%12
        if res >= old.month:
            #We have to go back an extra year
            year = year - 1
            month = 12 + old.month - res
        else:
            month = old.month - res
        result = datetime(year,month,old.day)
    return result

def grabForward(target,domain):
    #Get the closest datetime to the target dt, in the domain time series indexed by the datetimess.
    tmp = domain[target:]
    try:
        return tmp.iloc[0,:].values
    except:
        return tmp.iloc[0]

def grabNearest(target,domain,output=False):
    #Get the colest datetime to the target dt.
    #Domain has to a datetime indexed Series or DataFrame.
    left = domain[:target]
    right = domain[target:]
    if len(left) == 0:
        tt = right.index[0]
    elif len(right) == 0:
        tt = left.index[-1]
    else:
        candidate = [right.index[0],left.index[-1]]
        dev = [abs(x-target) for x in candidate]
        if dev[0] <= dev[1]:
            tt = candidate[0]
        else:
            tt = candidate[1]
    if output == True:
        print("@Function [grabNearest] grabbed tt at %s"%tt)
    if isinstance(domain,pd.Series):
        return domain[tt]
    elif isinstance(domain,pd.DataFrame):
        return domain[tt].values
    else:
        raise ValueError

#Configuration
start = datetime(1990,1,1)
end = datetime(2002,1,1)
regressStart = datetime(1990,1,1)
regressEnd = datetime(2008,1,1)
pin_date = 15      #The standing date for month frequecy data entry.
mode = "OLS"

#### Get the output gap time series

#Data inquiry
otpt_raw = pd.read_excel("data/production.xlsx")
otpt = pd.Series(otpt_raw.iloc[:,1].values,index=otpt_raw.iloc[:,0].values,name="industrial_output")[start:end]

#Fitting the quadratic model
regress = pd.DataFrame({'output':otpt})
regress['time'] = np.arange(len(regress))
regress['timesq'] = regress['time']**2
regress['constant'] = 1
model = sm.OLS(regress['output'],regress.drop('output',axis=1))
gapResult = model.fit(cov_type="HC3")
#Get the residual series
output_gap = gapResult.resid
output_gap.index = pd.to_datetime(output_gap.index)
output_gap.name = "outputgap"

#Set the common time line
timeline = output_gap.index

#### Get the inflation time series

#Data inquiry
infl_raw = pd.read_excel("data/inflation.xlsx",skiprows=3)
cpi = pd.Series(infl_raw.iloc[:,1].values,index=infl_raw.iloc[:,0].values,name="cpi")[flashBack(start,1):end]
infl = cpi.pct_change().dropna()

#### Get the federal funds rate

#Data inquiry
iff_raw = pd.read_excel("data/FEDFUNDS.xls",skiprows=10)
iff_tmp = pd.Series(iff_raw['FEDFUNDS'].values,index=iff_raw['observation_date'].values,name = "iff")

#Shaping the series, awkward.
iff1 = pd.Series(np.nan,index=timeline)
for time in timeline:
    iff1[time] = grabForward(target=time,domain=iff_tmp)
head = flashBack(timeline[0],1)
iff1[head] = grabForward(head,iff_tmp)
iff1 = iff1.sort_index()
iff = iff1[iff1.index[1:]]
iff_lag = iff1.shift().dropna()
iff.name = "iffrate"
iff_lag.name = "iffratelag"

#### Run the Taylor's model.

regress = pd.DataFrame([iff,output_gap,infl,iff_lag]).T
regress['Constant'] = 1
taylorModel = sm.OLS(regress.iffrate,regress.drop('iffrate',axis=1))
taylorResult = taylorModel.fit(cov_type="HC3")

#### Peel out the monetary shock of US

monetaryShock = taylorResult.resid
monetaryShock.name = "monetaryShock"

### Part II : Get the UIP deviation estimation

"""The way to estimate the deviation from UIP equation is to use a forward difference of spot exchange rate"""

#Data inquiry
exch_raw = pd.read_excel("data/exchange_rate.xlsx")
exch = pd.Series(exch_raw["US $ TO UK £ (WMR) - EXCHANGE RATE"].values,index=exch_raw['Name'].values,name="PoundDollarPrice")

#Shaping the p_t series
pt = pd.Series(np.nan,index=timeline,name="UIPdev")
for time,value in pt.iteritems():

    pt[time] = grabNearest(target=datetime(time.year+1,time.month,time.day),domain=exch) - grabNearest(time,exch)

#Regress pt on monetaryShock
if not (pt.index == monetaryShock.index).all(): #Check the index
    raise ValueError
indep = pd.DataFrame({"monetaryShock":monetaryShock,"Constant":1})
model1 = sm.OLS(pt,indep)
if mode == "HAC":
    result1 = model1.fit(cov_type="HAC",cov_kwds={"maxlags":1})
else:
    result1 = model1.fit()
print(result1.summary())

#Adding a exchange rate lag term
exch_lag = exch - exch.shift(1)
exch_lag = exch_lag[start:end]
intervn = pd.Series([grabNearest(x,exch_lag) for x in timeline],index=timeline,name="intervention")
indep["intervention"] = intervn
model2 = sm.OLS(pt,indep)
if mode == "HAC":
    result2 = model2.fit(cov_type="HAC",cov_kwds={'maxlags':1})
else:
    result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                 UIPdev   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     6.714
Date:                Tue, 27 Nov 2018   Prob (F-statistic):             0.0106
Time:                        20:51:37   Log-Likelihood:                 88.983
No. Observations:                 144   AIC:                            -174.0
Df Residuals:                     142   BIC:                            -168.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
monetaryShock     0.1545      0.060      2.591

In [ ]:
dataset = indep
dataset['UIPdev'] =pt
dataset["exchgrt"] = pd.Series([grabNearest(x,exch) for x in timeline],index=timeline,name="exchgrt")

In [ ]:
dataset.to_stata("dataset.dta")

In [ ]:
mode == "HAC"